In [1]:
# Import our dependencies
import pandas as pd
import xgboost as xgb
import joblib
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from pathlib import Path
from imblearn.over_sampling import SMOTE

In [2]:
import psycopg2
from config import db_password

# Establish a connection to the database by creating a cursor object
# The PostgreSQL server must be accessed through the PostgreSQL APP or Terminal Shell

conn = psycopg2.connect(host="localhost", port = 5432, database="baseball_data", user="postgres", password=db_password)

In [3]:
# Create a cursor object
cur = conn.cursor()

In [ ]:
# A sample query of all data from the "career_batter" table in the "baseball_data" database
cur.execute("""SELECT * FROM career_batter LIMIT 5""")
query_results = cur.fetchall()
print(query_results)

In [4]:
# import entire career_batter table from postgres to dataframe
career_batter = pd.read_sql('SELECT * FROM career_batter', conn)
career_batter

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,HBP,SH,SF,AVG,OBP,SLG
0,aardsda01,331,4,0,0,0,0,0,0.0,0.0,0,2.0,0.0,1.0,0.0,0.000000,0.000000,0.000000
1,aaronha01,3298,12364,2174,3771,624,98,755,2297.0,240.0,1402,1383.0,32.0,21.0,121.0,0.304998,0.373949,0.554513
2,aaronto01,437,944,102,216,42,6,13,94.0,9.0,86,145.0,0.0,9.0,6.0,0.228814,0.291506,0.327331
3,aasedo01,448,5,0,0,0,0,0,0.0,0.0,0,3.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,abadan01,15,21,1,2,0,0,0,0.0,0.0,4,5.0,0.0,0.0,0.0,0.095238,0.240000,0.095238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19893,zupofr01,16,18,3,3,1,0,0,0.0,0.0,2,6.0,0.0,0.0,0.0,0.166667,0.250000,0.222222
19894,zuvelpa01,209,491,41,109,17,2,2,20.0,2.0,34,50.0,2.0,18.0,0.0,0.221996,0.275142,0.276986
19895,zuverge01,266,142,5,21,2,1,0,7.0,0.0,9,39.0,0.0,16.0,0.0,0.147887,0.198675,0.176056
19896,zwilldu01,366,1280,167,364,76,15,30,202.0,46.0,128,155.0,4.0,31.0,0.0,0.284375,0.351275,0.437500


In [5]:
# import entire hall_batter table from postgres to dataframe
hall_batter = pd.read_sql('SELECT * FROM hall_batter', conn)
hall_batter

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,HBP,SH,SF,AVG,OBP,SLG,inducted
0,aardsda01,331.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,N
1,aaronha01,3298.0,12364.0,2174.0,3771.0,624.0,98.0,755.0,2297.0,240.0,1402.0,1383.0,32.0,21.0,121.0,0.304998,0.373949,0.554513,Y
2,aaronto01,437.0,944.0,102.0,216.0,42.0,6.0,13.0,94.0,9.0,86.0,145.0,0.0,9.0,6.0,0.228814,0.291506,0.327331,N
3,aasedo01,448.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,N
4,abadan01,15.0,21.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.095238,0.240000,0.095238,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17517,zupcibo01,319.0,795.0,99.0,199.0,47.0,4.0,7.0,80.0,7.0,57.0,137.0,6.0,20.0,8.0,0.250314,0.302540,0.345912,N
17518,zupofr01,16.0,18.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,6.0,0.0,0.0,0.0,0.166667,0.250000,0.222222,N
17519,zuvelpa01,209.0,491.0,41.0,109.0,17.0,2.0,2.0,20.0,2.0,34.0,50.0,2.0,18.0,0.0,0.221996,0.275142,0.276986,N
17520,zuverge01,266.0,142.0,5.0,21.0,2.0,1.0,0.0,7.0,0.0,9.0,39.0,0.0,16.0,0.0,0.147887,0.198675,0.176056,N


In [6]:
cur.close()
conn.close()

In [7]:
# ML model data preprocessing
# Convert inducted column to Y = 1, N = 0

b = {'Y': 1, 'N': 0}
hall_batter['inducted'] = hall_batter['inducted'].map(b).fillna(hall_batter['inducted'])
hall_batter

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,BB,SO,HBP,SH,SF,AVG,OBP,SLG,inducted
0,aardsda01,331.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.000000,0.000000,0.000000,0
1,aaronha01,3298.0,12364.0,2174.0,3771.0,624.0,98.0,755.0,2297.0,240.0,1402.0,1383.0,32.0,21.0,121.0,0.304998,0.373949,0.554513,1
2,aaronto01,437.0,944.0,102.0,216.0,42.0,6.0,13.0,94.0,9.0,86.0,145.0,0.0,9.0,6.0,0.228814,0.291506,0.327331,0
3,aasedo01,448.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0
4,abadan01,15.0,21.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.095238,0.240000,0.095238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17517,zupcibo01,319.0,795.0,99.0,199.0,47.0,4.0,7.0,80.0,7.0,57.0,137.0,6.0,20.0,8.0,0.250314,0.302540,0.345912,0
17518,zupofr01,16.0,18.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,6.0,0.0,0.0,0.0,0.166667,0.250000,0.222222,0
17519,zuvelpa01,209.0,491.0,41.0,109.0,17.0,2.0,2.0,20.0,2.0,34.0,50.0,2.0,18.0,0.0,0.221996,0.275142,0.276986,0
17520,zuverge01,266.0,142.0,5.0,21.0,2.0,1.0,0.0,7.0,0.0,9.0,39.0,0.0,16.0,0.0,0.147887,0.198675,0.176056,0


In [8]:
# Set target and features variables
y = hall_batter.inducted
X = hall_batter.drop(columns=["playerID", "inducted"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#Scale the data
#Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Use SMOTE to addres imbalanced dataset
X_resampled, y_resampled = SMOTE(random_state=42,
sampling_strategy='auto').fit_resample(
   X_train_scaled, y_train)

In [10]:
# Create a RFC model
XGB_model = XGBClassifier(max_depth = 5, min_child_weight = 1, gamma = 0, subsample = 0.8, colsample_bytree = 0.8, scale_pos_weight = 1, use_label_encoder = False)

# Fit the model
XGB_model = XGB_model.fit(X_resampled, y_resampled)

print(XGB_model)

[11:32:14] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)


In [11]:
# Use model to generate predictions
predictions = XGB_model.predict(X_test_scaled)

predictions

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
# Evaluate the model

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index= ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

# Calculate the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Display results
print("Confustion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confustion Matrix


,Predicted 0,Predicted 1
Actual 0,4307,35
Actual 1,9,30


Accuracy Score : 0.9899566309061858
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4342
           1       0.46      0.77      0.58        39

    accuracy                           0.99      4381
   macro avg       0.73      0.88      0.79      4381
weighted avg       0.99      0.99      0.99      4381



In [13]:
# Calculate feature importance in the RFC model
importances = XGB_model.feature_importances_
importances

array([0.00731292, 0.00526069, 0.7182625 , 0.14279458, 0.0062605 ,
       0.00430944, 0.00500301, 0.0284054 , 0.00394167, 0.01266681,
       0.00732481, 0.00399585, 0.0064075 , 0.02836067, 0.01121513,
       0.00417953, 0.00429905], dtype=float32)

In [14]:
# Sort the features by their importance
sorted(zip(XGB_model.feature_importances_, X.columns), reverse=True)

[(0.7182625, 'R'),
 (0.14279458, 'H'),
 (0.0284054, 'RBI'),
 (0.028360672, 'SF'),
 (0.01266681, 'BB'),
 (0.011215133, 'AVG'),
 (0.007324805, 'SO'),
 (0.007312917, 'G'),
 (0.006407495, 'SH'),
 (0.0062604975, '2B'),
 (0.00526069, 'AB'),
 (0.0050030085, 'HR'),
 (0.0043094424, '3B'),
 (0.0042990474, 'SLG'),
 (0.0041795326, 'OBP'),
 (0.00399585, 'HBP'),
 (0.0039416696, 'SB')]

In [15]:
# Save the Model
filename = 'Final_XGB_Batter_Model.sav'
joblib.dump(XGB_model, filename)

['Final_XGB_Batter_Model.sav']